# Advent of Code

This notebook contains my solutions for the 2020 version of [Advent of Code](https://adventofcode.com/2020).

### Imports and Dataimport

In [2]:
from itertools   import count
from typing      import Tuple, List

import re

In [4]:
def data(day: int, parser=str, sep='\n') -> list:
    "Split the day's input file into sections separated by `sep`, and apply `parser` to each."
    with open(f'data/day{day}.txt') as f:
        sections = f.read().rstrip().split(sep)
        return list(map(parser, sections))

## Day 1: Report Repair

### Part 1
The first challenge is to find two numbers in a list that add up to 2020 and return their product.

In [6]:
test1_input = [1721, 979, 366, 299, 675, 1456]
# 1721 + 299 = 2020 and 1721 * 299 = 514579
test1_1output = 514579

# simple brute force approach with nested for loops and quadratic runtime complexity in relation to the input size
def day1_1dumb(nums):
    for x in nums:
        for y in nums:
            if (y == 2020 - x):
                return x * y

def day1_1(nums):
    complements = set()
    for x in nums:
        y = 2020 - x
        
        if x in complements:
            return x * y
        
        complements.add(y)
        
assert day1_1dumb(set(test1_input)) == test1_1output
assert day1_1(set(test1_input)) == test1_1output

input1 = set(data(1, int))
day1_1(input1)

926464

### Part 2
The second part of the puzzle is to find the three distinct numbers that add to 2020 and return their product.

In [7]:
test1_input = [1721, 979, 366, 299, 675, 1456]
# 979 + 366 + 675 = 2020 and 979 * 366 * 675 = 241861950
test1_2output = 241861950

# simple brute force approach with three nested for loops and cubic runtime complexity in relation to the input size
def day1_2dumb(nums):
    for x in nums:
        for y in nums:
            for z in nums:
                if (z == 2020 - y - x):
                    return x * y * z

def day1_2(nums):
    for x in nums:
        complements = set()
        yz = 2020 - x

        for y in nums:
            z = yz - y

            if y in complements:
                return x * y * z

            complements.add(z)

assert day1_2dumb(set(test1_input)) == test1_2output
assert day1_2(set(test1_input)) == test1_2output

input1 = set(data(1, int))
day1_2(input1)

65656536

### Benchmark

In [8]:
%timeit day1_1dumb(input1)

1.37 ms ± 75.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [9]:
%timeit day1_1(input1)

22.7 µs ± 233 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [10]:
%timeit day1_2dumb(input1)

123 ms ± 8.84 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
%timeit day1_2(input1)

988 µs ± 48.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Day 2: Password Philosophy

### Part 1
The First part of this days challenge is to count how many passwords are valid according to their policies.

A given password string is considered valid if it contains between min and max instances of a specific character, where min, max and the character are specified in the policy.

In [13]:
test2_input1 = '1-3 a: abcde'
test2_input2 = '1-3 b: cdefg'
test2_input3 ='2-9 c: ccccccccc'
test2_1output1 = True
test2_1output2 = False
test2_1output3 = True

Pw_Policy = Tuple[int, int, str, str]

def parse_pw_policy(line: str) -> Pw_Policy:
    "Given '1-3 b: cdefg', return (1, 3, 'b', 'cdefg')."
    mmin, mmax, letter, pw = re.findall(r'[^-:\s]+', line)
    return (int(mmin), int(mmax), letter, pw)

def check_pw(policy) -> bool:
    mmin, mmax, letter, pw = policy
    return mmin <= pw.count(letter) <= mmax

assert check_pw(parse_pw_policy(test2_input1)) == test2_1output1
assert check_pw(parse_pw_policy(test2_input2)) == test2_1output2
assert check_pw(parse_pw_policy(test2_input3)) == test2_1output3

input2: List[Tuple] = data(2, parse_pw_policy)

valid = 0
for pw_string in list(map(check_pw,input2)):
    if pw_string == True:
        valid += 1
valid

378

### Part 2
The second part of this puzzle changes the interpretation of the password policy.
This digits that used to express min and max before now denote a position in the password string, however the index starts at 1.

A password is considered valid with this new policy if it has the specified letter at exactly one of the two given positions. How many passwords are valid?

In [16]:
test2_2output1 = True
test2_2output2 = False
test2_2output3 = False

def check_pw2(policy, offset=1) -> bool:
    first, second, letter, pw = policy
    return (pw[first - offset] == letter) ^ (pw[second - offset] == letter)

assert check_pw2(parse_pw_policy(test2_input1)) == test2_2output1
assert check_pw2(parse_pw_policy(test2_input2)) == test2_2output2
assert check_pw2(parse_pw_policy(test2_input3)) == test2_2output3

valid = 0
for pw_string in list(map(check_pw2,input2)):
    if pw_string == True:
        valid += 1
valid

280

## Day 3: Toboggan Trajectory

### Part 1
This challenge provides a map that contains '.' for free spaces and '#' for trees.

The questions is how many trees are encountered for a given map with a path that takes 3 steps right and 1 down until the bottom of the map is reached with the assumption that the pattern specified in the map repeats infinitely to the right.

In [17]:
test3_input = ['..##.......',
               '#...#...#..',
               '.#....#..#.',
               '..#.#...#.#',
               '.#...##..#.',
               '..#.##.....',
               '.#.#.#....#',
               '.#........#',
               '#.##...#...',
               '#...##....#',
               '.#..#...#.#']
test3_1output = 7

Grid = List[str]

def count_trees_along_slope(grid, dx=3, dy=1, tree='#'):
    height = len(grid)
    width = len(grid[0])
    trees = 0
    col = 0
    for row, col in zip(range(0, height, dy), count(0, dx)):
        if grid[row][col % width] == tree:
            trees += 1
    return trees

assert count_trees_along_slope(test3_input, dx=3, dy=1, tree='#') == test3_1output

input3 : Grid = data(3)

count_trees_along_slope(input3)

220

### Part2
The second challenge for this day asks for the product of the encountered trees for paths with different slopes.

In [18]:
test3_2output = 336

def count_trees_along_all_slopes(grid):
    def t(dx, dy):
        return count_trees_along_slope(grid, dx, dy)
    return t(1, 1) * t(3, 1) * t(5, 1) * t(7, 1) * t(1, 2)

assert count_trees_along_all_slopes(test3_input) == test3_2output

count_trees_along_all_slopes(input3)

2138320800